# 实现一个包含记忆的Chatbot:

## 1. 回顾
简单回顾下[tutorial_langchain.ipynb](../../tutorial_langchain.ipynb)中关于chatbot的最基础实现，即一个chat model和prompt template的定义，可以选择用chain的方式连接起来。

In [1]:
from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv
# 读取 .env 文件中的环境变量, 我们可以在 .env 文件中设置 OPENAI_API_KEY，文件内容类似于： OPENAI_API_KEY=your_api_key
load_dotenv() 
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv('OPENAI_API_BASE')

# 调用langchain的封装init_chat_model方法，初始化一个chat模型
# 这里我用的是星火x1模型，x1兼容openai的调用格式，所以可以直接调用openai的接口
# base_url和api_key是从.env 文件中读取的, 来自星火开放平台
model = init_chat_model(model = "x1", model_provider="openai",
                        base_url=base_url, api_key=api_key)

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(
            "你好，我是{name}"
        ),
        AIMessagePromptTemplate.from_template(
            "你好, 有什么可以帮助的？"
        ),
        HumanMessagePromptTemplate.from_template(
            "我的名字是什么"
        ),
    ]
)

parse = StrOutputParser()
chain = prompt_template | model | parse
chain.invoke({"name": "Retro"})


'你的名字是Retro呀！很高兴认识你~ (◕ᴗ◕✿)'

## 2. 历史记忆（History Memory）管理
ref: 
- [How to add message history](https://python.langchain.com/docs/how_to/message_history/)
- [How to add memory to chatbots](https://python.langchain.com/docs/how_to/chatbots_memory/#message-passing)

在构建chatbot时，将会话状态传入和传出chain是至关重要的。LangGraph实现了一个内置的持久化层（persistence layer），允许chain状态自动持久化到内存或外部后端（如SQLite、Postgres或Redis）。详细信息可以在LangGraph [persistence documentation](https://langchain-ai.github.io/langgraph/how-tos/persistence/?_gl=1*15xmk3c*_ga*MTE5OTY2ODY1MC4xNzQ3MjM2ODQ2*_ga_47WX3HKKY2*czE3NDc2NjA1NzkkbzE4JGcxJHQxNzQ3NjYzMzgxJGowJGwwJGgw)中找到。

接下来我们会通过LangGraph的StateGraph类，实现一个拥有自动历史messages管理的chatbot，来展示LangGraph是怎么做历史记忆管理的。
- 这里所谓的state就是用于存储历史记忆的。由于chat model通常接受message列表作为输入，并且输出一个message，所以我们可以让state来保存历史messages，即维护一个messages state。可以使用LangGraph内置的MessagesState类来实现这个需求。
- 至于具体的持久化方式，简单起见，这里我们选择持久化到内存，可以使用MemorySaver，这是一个in-memory checkpoint saver，需要在graph编译时传入。



按照步骤，我们
1. 定义一个StateGraph，并将该graph的state定义为message列表（用MessagesState）；
2. 定义chat model的调用函数call_model，这个函数输入是一个state，然后将state中的具体数据（这里是history messages）作为输入传给chat model，返回model的输出；
3. 往graph中添加一个node，这个node将会调用call_model；
4. 用一个内存中的checkpointer来编译graph，从而在每次运行时存储messages。

In [3]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 定义一个StateGraph，并将该graph的state定义为message列表（用MessagesState）；
workflow = StateGraph(state_schema=MessagesState)


# 定义调用chat的函数，根据输入的state（本质上是message列表），调用chat model并返回response；
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


# 往graph中添加一个node，这个node将会调用chat model；
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# 用一个内存中的checkpointer来编译graph，从而在每次运行时存储messages。
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

当我们运行该app时，我们传入一个指定thread_id的配置字典dict。此ID用于区分会话线程（例如，在不同用户之间）。

In [4]:
config = {"configurable": {"thread_id": "aaa666"}}
query = "你好，我叫Retro"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

你好，Retro！很高兴认识你～ 今天有什么想和我分享，或者需要帮忙的地方吗？无论是问题、话题，还是随便聊聊，我都在这里哦！ 😊


In [5]:
query = "我的名字是什么?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是 **Retro** 呀！😄 有没有什么想和我聊聊的？无论是生活、兴趣，还是任何问题，都可以尽管说哦～


In [6]:
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

你好，我叫Retro
================================== Ai Message ==================================

你好，Retro！很高兴认识你～ 今天有什么想和我分享，或者需要帮忙的地方吗？无论是问题、话题，还是随便聊聊，我都在这里哦！ 😊
================================ Human Message =================================

我的名字是什么?
================================== Ai Message ==================================

你的名字是 **Retro** 呀！😄 有没有什么想和我聊聊的？无论是生活、兴趣，还是任何问题，都可以尽管说哦～


该graph的state历史可以通过`.get_state`得到：

In [7]:
state = app.get_state(config).values

for message in state["messages"]:
    message.pretty_print()

================================ Human Message =================================

你好，我叫Retro
================================== Ai Message ==================================

你好，Retro！很高兴认识你～ 今天有什么想和我分享，或者需要帮忙的地方吗？无论是问题、话题，还是随便聊聊，我都在这里哦！ 😊
================================ Human Message =================================

我的名字是什么?
================================== Ai Message ==================================

你的名字是 **Retro** 呀！😄 有没有什么想和我聊聊的？无论是生活、兴趣，还是任何问题，都可以尽管说哦～
